### Anime ratingsien perusteella suosittelumalli käyttäen SVD-metodia.
Ajatus, tehdään malli samplella.
Mallin voi ajaa kenelle tahansa, jolla user_id.

Otetaan sisään ratings data (käyttäjät, anime_id ja arviot) ja anime data (anime_id ja kuvaukset)

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
ratings_data = pd.read_csv("rating.csv")
anime_data = pd.read_csv("anime.csv")

Datan tutkimista ja muokkaamista

In [84]:
ratings_data.head(10), 

(   user_id  anime_id  rating
 0        1        20      -1
 1        1        24      -1
 2        1        79      -1
 3        1       226      -1
 4        1       241      -1
 5        1       355      -1
 6        1       356      -1
 7        1       442      -1
 8        1       487      -1
 9        1       846      -1,)

In [85]:
user_rating_counts = ratings_data.groupby("user_id")["rating"].count()
# Display the distribution
print(user_rating_counts.sort_values(ascending=False).head(5))

user_id
48766    10227
42635     3747
53698     2905
57620     2702
59643     2633
Name: rating, dtype: int64


In [86]:
# # movies_data["genre"].unique()
# unique_genres = list(anime_data["genre"].unique())
# print(unique_genres)
# unique_genres

In [87]:
genre_counts = anime_data["genre"].value_counts()
print(genre_counts)

genre
Hentai                                                  823
Comedy                                                  523
Music                                                   301
Kids                                                    199
Comedy, Slice of Life                                   179
                                                       ... 
Adventure, Drama, Fantasy, Game, Sci-Fi                   1
Adventure, Demons, Fantasy, Historical                    1
Action, Comedy, Drama, Mecha, Music, Sci-Fi, Shounen      1
Action, Comedy, Fantasy, Mecha, Sci-Fi, Shounen           1
Hentai, Slice of Life                                     1
Name: count, Length: 3264, dtype: int64


Poistetaan -1 ratings-arvot, oletus kuvastavat sitä, että reittausta ei ole annettu

In [88]:
ratings_data = ratings_data[ratings_data.rating!=-1]

In [89]:
ratings_data.shape

(6337241, 3)

In [90]:
ratings_data.head()

,user_id,anime_id,rating
47,1,8074,10
81,1,11617,10
83,1,11757,10
101,1,15451,10
153,2,11771,10


### Mallinnus

In [91]:
from surprise import Dataset
from surprise import Reader
 
# Get minimum and maximum rating from the dataset
min_rating = ratings_data.rating.min()
max_rating = ratings_data.rating.max()
 
print(min_rating)
print(max_rating)

1
10


Otetaan 1000 otos mallinnusta varten

In [92]:
ratings_data_s = ratings_data.sample(n=1000, random_state=42).reset_index(drop=True)  # Sample 6000 rows

In [93]:
ratings_data_s.head()

,user_id,anime_id,rating
0,53103,6707,7
1,32884,339,8
2,24907,23755,9
3,52213,10156,7
4,994,1562,10


Etsitään mallin käsin testausta varten user_id,jota ei ole sample_datassa

In [94]:
ratings_data_s[ratings_data_s["user_id"] == 47]


,user_id,anime_id,rating


Varmistetaan samplen koko

In [95]:
ratings_data_s.shape

(1000, 3)

Vasrinainen mallin ajaminen

In [96]:
reader = Reader(rating_scale=(min_rating, max_rating))
data = Dataset.load_from_df(ratings_data_s[['user_id', 'anime_id', 'rating']], reader)

In [97]:
from surprise import SVD
from surprise.model_selection import cross_validate
 
svd = SVD(n_epochs=10)
results = cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=10, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 10 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Fold 6  Fold 7  Fold 8  Fold 9  Fold 10 Mean    Std     
RMSE (testset)    1.7532  1.5975  1.3649  1.4165  1.3933  1.3274  1.5350  1.4812  1.4463  1.2617  1.4577  0.1354  
MAE (testset)     1.3555  1.2465  1.1032  1.1114  1.0822  1.0004  1.1395  1.1567  1.1097  0.9998  1.1305  0.1014  
Fit time          0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.01    0.00    
Test time         0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    0.00    


In [98]:
print("Average MAE: ", np.average(results["test_mae"])) # Absuluuttinen ero ennustetun ja todellisen reittauksen välillä, mitä pienempi, sen parempi
print("Average RMSE: ", np.average(results["test_rmse"])) # Erojen neliö, mitä pienempi sen parempi

Average MAE:  1.1304932128471816
Average RMSE:  1.457694572359773


MAEn ja RMSE:n pitäisi ilmeisesti mielellään olla alle 1, mutta tässä asteikko 1-10 eikä 1-5,
joten ehkä tämä ero kuitekin ok, vaikkei erinomainen

Mallin tuunaus: Hyperparameter Tuning using grd search cross-validation

In [99]:
from surprise.model_selection import GridSearchCV
 
param_grid = {
  'n_factors': [20, 50, 100],
  'n_epochs': [5, 10, 20]
}
 
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=10)
gs.fit(data)
 
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.4508692038992315
{'n_factors': 50, 'n_epochs': 20}


Tulos parani hiukan, muttei merkittävästi

In [100]:
from surprise.model_selection import train_test_split
 
# best hyperparameters
best_factor = gs.best_params['rmse']['n_factors']
best_epoch = gs.best_params['rmse']['n_epochs']
 
# sample random trainset and testset
# test set is made of 20% of the ratings.
trainset, testset = train_test_split(data, test_size=.20)
 
# We'll use the famous SVD algorithm.
svd = SVD(n_factors=best_factor, n_epochs=best_epoch)
 
# Train the algorithm on the trainset
svd.fit(trainset)

Mallin ajaminen valitulle user_id:lle, jos ymmärrän oikein.
User_id:n ei tarvitse olla sample-datassa
Tässä esimerkkinä user_id : 47.
User_id:lle annetaan top 10 suositusta.

In [101]:
def generate_recommendation(model, user_id, ratings_df, movies_df, n_items):
   # Get a list of all movie IDs from dataset
   movie_ids = ratings_df["anime_id"].unique()
 
   # Get a list of all movie IDs that have been watched by user
   movie_ids_user = ratings_df.loc[ratings_df["user_id"] == user_id, "anime_id"]
    # Get a list off all movie IDS that that have not been watched by user
   movie_ids_to_pred = np.setdiff1d(movie_ids, movie_ids_user)
 
   # Apply a rating of 4 to all interactions (only to match the Surprise dataset format)
   test_set = [[user_id, movie_id, 4] for movie_id in movie_ids_to_pred]
 
   # Predict the ratings and generate recommendations
   predictions = model.test(test_set)
   pred_ratings = np.array([pred.est for pred in predictions])
   print("Top {0} item recommendations for user {1}:".format(n_items, user_id))
   # Rank top-n movies based on the predicted ratings
   index_max = (-pred_ratings).argsort()[:n_items]
 
   for i in index_max:
       movie_id = movie_ids_to_pred[i]
      
       print(movies_df[movies_df["anime_id"]==movie_id]["name"].values[0], pred_ratings[i])
       print(movies_df[movies_df["anime_id"]==movie_id]["genre"].values[0])
    

# define which user ID that we want to give recommendation
userID = 47  
# define how many top-n movies that we want to recommend
n_items = 10
# generate recommendation using the model that we have trained
generate_recommendation(svd,userID,ratings_data,anime_data,n_items)

Top 10 item recommendations for user 47:
Cowboy Bebop 8.404794775685787
Action, Adventure, Comedy, Drama, Sci-Fi, Space
Clannad 8.359785922872902
Comedy, Drama, Romance, School, Slice of Life, Supernatural
Steins;Gate 8.308564632506423
Sci-Fi, Thriller
Death Note 8.248094296060046
Mystery, Police, Psychological, Supernatural, Thriller
Usagi Drop 8.239677844579711
Josei, Slice of Life
Shokugeki no Souma 8.23728066881738
Ecchi, School, Shounen
Blood-C 8.226461617523643
Action, Horror, School, Supernatural, Vampire
Yamato Nadeshiko Shichihenge♥ 8.181443796586825
Comedy, Shoujo
Clannad: Mou Hitotsu no Sekai, Tomoyo-hen 8.175111787139446
Drama, Romance, School, Slice of Life
Monster 8.1729449693308
Drama, Horror, Mystery, Police, Psychological, Seinen, Thriller


User_id:n tietojen hakeminen, 10 parhaiten reitattua

In [102]:
def top_rated_movies(user_id, ratings_data, anime_data, n_items=10):
    """
    Lists the top highest-rated movies for a given user and retrieves their name and genre.
    
    Parameters:
        user_id (int): ID of the user to filter ratings for.
        ratings_data (pd.DataFrame): DataFrame containing user-item ratings with columns ['user_id', 'anime_id', 'rating'].
        movies_data (pd.DataFrame): DataFrame containing movie details with at least ['anime_id', 'name', 'genre'].
        n_items (int): Number of top-rated movies to retrieve (default is 10).
    
    Returns:
        pd.DataFrame: DataFrame with top-rated movies including their name, genre, and rating.
    """
    # Filter ratings for the given user
    user_ratings = ratings_data[ratings_data["user_id"] == user_id]
    
    # Get the top n_items highest-rated movies
    top_rated = user_ratings.nlargest(n_items, "rating")
    
    # Merge with movies_df to get the name and genre
    top_rated = top_rated.merge(anime_data, on="anime_id", how="inner")
    
    # Select relevant columns
    result = top_rated[["anime_id", "name", "genre", "rating_x"]].sort_values(by="rating_x", ascending=False)
    
    return result

# Example usage
# top_movies = top_rated_movies(user_id=1, ratings_df=ratings_data, movies_df=movies_data, n_items=10)
# print(top_movies)

user_id = 47 

top_rated_movies(user_id,ratings_data, anime_data, n_items=10)


,anime_id,name,genre,rating_x
0,120,Fruits Basket,"Comedy, Drama, Fantasy, Romance, Shoujo, Slice...",10
1,199,Sen to Chihiro no Kamikakushi,"Adventure, Drama, Supernatural",10
2,523,Tonari no Totoro,"Adventure, Comedy, Supernatural",10
3,853,Ouran Koukou Host Club,"Comedy, Harem, Romance, School, Shoujo",10
4,3588,Soul Eater,"Action, Adventure, Comedy, Fantasy, Shounen, S...",10
5,5681,Summer Wars,"Comedy, Sci-Fi",10
6,6702,Fairy Tail,"Action, Adventure, Comedy, Fantasy, Magic, Sho...",10
7,7337,"Hetalia Axis Powers Movie: Paint it, White","Comedy, Historical, Parody",10
8,431,Howl no Ugoku Shiro,"Adventure, Drama, Fantasy, Romance",9
9,530,Bishoujo Senshi Sailor Moon,"Demons, Magic, Romance, Shoujo",9
